So far we've learnt how to scrape the web, and how to make a request for information from an API. Some websites make APIs even easier. Check out [RapidAPI](https://rapidapi.com/) they take care of writing most of the code for you.

We will use the [AeroDataBox API](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/), which can retrieve all sorts of information about flights and airports. We will show you how to retrieve information about the airports, and then it's up to you to apply this, along with what you've already learnt this week, to **produce a function, which retrieves tomorrows flight information for the major airports in the cities you web scraped**.

In [ ]:
import pandas as pd
import requests

On the left hand side of the AeroDataBox API page, you'll see a list of options for information that you can retrieve:
> - Flights API
- Subsciption / PUSH API
- Airport API
- Aircraft API
- Healthcheck & Status API

1. We want to select `Airport API`

2. Then within Airport API we want to select `Search airports by location`

3. Now in the middle third you'll want to enter the `latitude` and `longitude` of any city to test... we chose Berlin: latitude 52.31 longitude 13.24. Next we changed the `radiusKM` to only 50km. And finally set `withFlightInfoOnly` to true, so it will only return airports which have flight data (scheduled or live) available.

4. On the right hand third of the screen you should see a block of code that looks pretty unfamiliar. This is because by default the code is probably set to *(Node.js) Axios*. However, we have the power to change this to familiar python. Select the dropdown box at the top of the code and select `python > requests`.

Now you can copy the code to your notebook and it should look a little something like the cell below:

In [ ]:
url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

querystring = {"lat":"52.31","lon":"13.24","radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "YOUR_API_KEY_HERE",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"searchBy":{"lat":52.31,"lon":13.24},"items":[{"icao":"EDDB","iata":"BER","name":"Berlin, Berlin Brandenburg","shortName":"Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"countryCode":"DE"}]}


Let's view the response as `.json()` instead of `.text` so that it's easier to read

In [ ]:
print(response.json)

<bound method Response.json of <Response [200]>>


In [ ]:
from IPython.display import JSON

In [ ]:
display(JSON(response.json()))

<IPython.core.display.JSON object>

We can now turn this into a dataframe using `.json_normalize()`

In [ ]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889


Let's now use this for the latitude and longitude of multiple cities

In [ ]:
lat = [52.3112, 51.3026]
lon = [13.2418, 0.739]

list_for_df_airports = []

for i in range(len(lat)):
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

    querystring = {"lat":lat[i],"lon":lon[i],"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

    headers = {
        "X-RapidAPI-Key": "YOUR_API_KEY_HERE",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    list_for_df_airports.append(response.json()['items'])

In [ ]:
# lat = [52.3112, 51.3026]
# lon = [13.2418, 0.739]

# list_for_df_airports = []

# for i in range(len(lat)):
#     url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat[i]}/{lon[i]}/km/50/10"

#     querystring = {"withFlightInfoOnly":"true"}

#     headers = {
#         "X-RapidAPI-Key": "YOUR_API_KEY_HERE",
#         "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
#     }

#     response = requests.request("GET", url, headers=headers, params=querystring)
    
#     list_for_df_airports.append(response.json()['items'])

In [ ]:
list_for_df_airports

[[{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin, Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE'}],
 [{'icao': 'EGMC',
   'iata': 'SEN',
   'name': 'Southend',
   'shortName': 'Southend',
   'municipalityName': 'Southend',
   'location': {'lat': 51.5714, 'lon': 0.695555},
   'countryCode': 'GB'}]]

In [ ]:
list_for_df_airports[1][0]['icao']

'EGMC'

In [ ]:
airpot_dict = {}
airpot_dict['icao'] = list_for_df_airports[0][0]['icao']
airpot_dict['name'] = list_for_df_airports[0][0]['name']

airpot_dict

{'icao': 'EDDB', 'name': 'Berlin, Berlin Brandenburg'}

In [ ]:
list_for_df = []

for i in list_for_df_airports:
    airpots_dict = {}
    
    airpots_dict['icao'] = i[0]['icao']
    airpots_dict['name'] = i[0]['name']
    list_for_df.append(airpots_dict)

In [ ]:
list_for_df

[{'icao': 'EDDB', 'name': 'Berlin, Berlin Brandenburg'},
 {'icao': 'EGMC', 'name': 'Southend'}]

In [ ]:
airports_df = pd.DataFrame(list_for_df)
airports_df

,icao,name
0,EDDB,"Berlin, Berlin Brandenburg"
1,EGMC,Southend


In [ ]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
  list_for_df = []

  for i in range(len(latitudes)):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/100/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "YOUR_API_KEY_HERE"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

###### **Challenge 1:** Icao codes
If you use the above for all of your cities, you can create a DataFrame of all the airports and their associated `icao` codes. Perfect for a relational database.

###### **Challenge 2:** Arrivals information
Using what you have been shown above, plus the skills you've learnt in the last couple of days:
1. In `AeroDataBox API` use the `Flight API` > `FIDS/Schedules: Airport departures and arrivals by airport ICAO code` section
2. Fill out the parameters in the middle third and then copy the `python: requests` code from the right hand third
3. Explore the data you get back. What would be useful in your DataFrame and what can be excluded? Remember Gans wants to know about when people are arriving in the city
4. Make a DataFrame from the information you see as important
5. Condense everything you did above into a function that can take a list of ICAO codes as an input, and as an output gives you a DataFrame with the information for *tomorrows arrivals*

In [ ]:


url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2022-10-04T20:00/2022-10-05T08:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": "YOUR_API_KEY_HERE",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"arrivals":[{"departure":{"airport":{"icao":"LEPA","iata":"PMI","name":"Palma De Mallorca"},"scheduledTimeLocal":"2022-10-04 17:20+02:00","actualTimeLocal":"2022-10-04 17:24+02:00","scheduledTimeUtc":"2022-10-04 15:20Z","actualTimeUtc":"2022-10-04 15:24Z","terminal":"N","checkInDesk":"002-017","gate":"C68","quality":["Basic","Live"]},"arrival":{"scheduledTimeLocal":"2022-10-04 20:05+02:00","scheduledTimeUtc":"2022-10-04 18:05Z","terminal":"0","quality":["Basic"]},"number":"FR 279","status":"Expected","codeshareStatus":"Unknown","isCargo":false,"aircraft":{"model":"Airbus A320"},"airline":{"name":"Ryanair"}},{"departure":{"airport":{"icao":"LGSA","iata":"CHQ","name":"Souda"},"scheduledTimeLocal":"2022-10-04 17:50+03:00","scheduledTimeUtc":"2022-10-04 14:50Z","quality":["Basic"]},"arrival":{"scheduledTimeLocal":"2022-10-04 20:00+02:00","scheduledTimeUtc":"2022-10-04 18:00Z","terminal":"0","quality":["Basic"]},"number":"FR 5642","status":"Unknown","codeshareStatus":"Unknown","isCargo":fa

In [ ]:
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [ ]:
from datetime import datetime, date, timedelta
from pytz import timezone

In [ ]:
today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
today

datetime.date(2023, 3, 29)

In [ ]:
tomorrow = (today + timedelta(days=1))
tomorrow           

datetime.date(2023, 3, 30)

In [ ]:
def tomorrows_flight_arrivals(icao_list):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))

  list_for_df = []

  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': "YOUR_API_KEY_HERE"
          }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        # .get() is another way of ensuring our code doesn't break
        # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
        # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        list_for_df.append(flights_dict)

  return pd.DataFrame(list_for_df)

In [ ]:
icaos = ['EDDB', 'EGLL']
tomorrows_flight_arrivals(icaos).tail(10)

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
2382,EGLL,2023-03-30 22:25+01:00,2,Istanbul,LTFM,None,Pakistan International,PK 5987,2023-03-29
2383,EGLL,2023-03-30 22:10+01:00,3,Chicago,KORD,None,Qatar Airways,QR 2858,2023-03-29
2384,EGLL,2023-03-30 22:00+01:00,4,Doha,OTHH,2023-03-30 16:40+03:00,Qatar Airways,QR 5,2023-03-29
2385,EGLL,2023-03-30 22:35+01:00,5,Glasgow,EGPF,None,Qatar Airways,QR 5905,2023-03-29
2386,EGLL,2023-03-30 22:00+01:00,2,Lisbon,LPPT,2023-03-30 19:20+01:00,Azores,S4 8048,2023-03-29
2387,EGLL,2023-03-30 22:25+01:00,2,Istanbul,LTFM,2023-03-30 20:20+03:00,Turkish,TK 1987,2023-03-29
2388,EGLL,2023-03-30 22:00+01:00,2,Lisbon,LPPT,2023-03-30 19:20+01:00,TAP Air Portugal,TP 1358,2023-03-29
2389,EGLL,2023-03-30 22:55+01:00,2,Lisbon,LPPT,2023-03-30 20:15+01:00,TAP Air Portugal,TP 1366,2023-03-29
2390,EGLL,2023-03-30 22:25+01:00,2,Istanbul,LTFM,None,United,UA 6918,2023-03-29
2391,EGLL,2023-03-30 22:00+01:00,4,Doha,OTHH,None,Oman Air,WY 6433,2023-03-29
